In [2]:
import pandas as pd
import numpy as np
from config import gkey
import numpy as np
import requests
import json
import gmaps

# Configure gmaps
gmaps.configure(api_key=gkey)

In [3]:
#url of national capitals for scraping
url = 'https://en.wikipedia.org/wiki/List_of_national_capitals_by_latitude'

#temperaturate anamolies from https://data.giss.nasa.gov/gistemp/
zonal_temp_anamolies = pd.read_csv('C:/Users/joelb/Documents/Github/ClimateChange/ZonAnn.Ts+dSST.csv')


In [4]:

# scrape html table from wikipediea
tables = pd.read_html(url)

#grabs the 2nd table on the webpage
city_by_hemisphere = tables[1]

#give the table column names
city_by_hemisphere.columns = ['City','Latitude','Country','Notes']

#drops the first row that is just headers
city_by_hemisphere = city_by_hemisphere.drop(0,axis=0)


#converts the latitude column into numeric
city_by_hemisphere['Latitude'] = city_by_hemisphere['Latitude'].str.replace('−','-')
city_by_hemisphere['Latitude'] = pd.to_numeric(city_by_hemisphere['Latitude'])

#if lat is negative its south, if positive its north
city_by_hemisphere['Hemisphere'] = np.where(city_by_hemisphere['Latitude']<=0, 'south', 'north')


In [5]:

#determies what zone of the hemisphere the city is in
city_by_hemisphere['zone'] = np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']>0,city_by_hemisphere['Latitude']<=24), \
                                            city_by_hemisphere['Hemisphere'] == 'north'),'EQU-24N', \
                             np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']>24,city_by_hemisphere['Latitude']<=44), \
                                            city_by_hemisphere['Hemisphere'] == 'north'),'24N-44N',
                             np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']>44,city_by_hemisphere['Latitude']<=64), \
                                            city_by_hemisphere['Hemisphere'] == 'north'),'44N-64N',
                             np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']>64,city_by_hemisphere['Latitude']<=90), \
                                            city_by_hemisphere['Hemisphere'] == 'north'),'64N-90N', \
                            np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']<0,city_by_hemisphere['Latitude']>=-24), \
                                            city_by_hemisphere['Hemisphere'] == 'south'),'24S-EQU', \
                            np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']<-24,city_by_hemisphere['Latitude']>=-44), \
                                            city_by_hemisphere['Hemisphere'] == 'south'),'44S-24S', \
                            np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']<-44,city_by_hemisphere['Latitude']>=-64), \
                                            city_by_hemisphere['Hemisphere'] == 'south'),'64S-44S', \
                            np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']<-64,city_by_hemisphere['Latitude']>=-90), \
                                            city_by_hemisphere['Hemisphere'] == 'south'),'64S-44S', \
                                     ""))))))))                                  


In [6]:
#get column headers for the year to be used later
year_header = list(zonal_temp_anamolies['Year'])

In [7]:
#transpose the dataframe for merging
zonal_temp_anamolies = zonal_temp_anamolies.transpose()


In [8]:

#merge the city capitals with the zonal temperature data
climate_df =  city_by_hemisphere.merge(zonal_temp_anamolies, how='left', left_on='zone', right_index=True)

In [9]:
#rename columns to the appropriate year
climate_df.rename(columns=dict(zip(climate_df.columns[6:],year_header)),inplace=True)

In [10]:
#drop a notes column that is not needed
climate_df = climate_df.drop('Notes',axis=1)



In [11]:

#sums temperature anomalies over a period of years.  will be the layers on the leaflet map
climate_df['rng_1880_1910'] = climate_df.iloc[:, 5:36].sum(axis='columns')
climate_df['rng_1911_1940'] = climate_df.iloc[:, 36:66].sum(axis='columns')
climate_df['rng_1941_1970'] = climate_df.iloc[:, 66:96].sum(axis='columns')
climate_df['rng_1971_2000'] = climate_df.iloc[:, 96:126].sum(axis='columns')
climate_df['rng_2001_2017'] = climate_df.iloc[:, 126:143].sum(axis='columns')

In [13]:
#create lat and lgn fields for google maps geocode api
climate_df["Lat"] = ""
climate_df["Lng"] = ""

#set parameters to my gkey
params = {"key": gkey}

In [14]:
count = 0
bad_list = []

for index, row in climate_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['City']
    country = row['Country']

    # update address key value
    params['address'] = f"{city},{country}"

    # make request, print url
    cities_lat_lng = requests.get(base_url, params=params)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:

        climate_df.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        climate_df.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

        count +=1
        print("num records processed" + str(count))
    except:
        print("didnt copy street: " +  "city: " + city )
        bad_list.append(city)
        

num records processed1
num records processed2
num records processed3
num records processed4
num records processed5
num records processed6
num records processed7
num records processed8
num records processed9
num records processed10
num records processed11
num records processed12
num records processed13
num records processed14
num records processed15
num records processed16
num records processed17
num records processed18
num records processed19
num records processed20
num records processed21
num records processed22
num records processed23
num records processed24
num records processed25
num records processed26
num records processed27
num records processed28
num records processed29
num records processed30
num records processed31
num records processed32
num records processed33
num records processed34
num records processed35
num records processed36
num records processed37
num records processed38
num records processed39
num records processed40
num records processed41
num records processed42
n

In [16]:
climate_df.to_excel('C:/Users/joelb/Documents/Github/ClimateChange/test.xlsx')

In [17]:
#drop original lat field so it does not conflict with google lat field
climate_df = climate_df.drop('Latitude',axis=1)


In [18]:
climate_df.columns = [str(i) for i in climate_df.columns.values.tolist()]
print(climate_df.columns.values)

['City' 'Country' 'Hemisphere' 'zone' '1880' '1881' '1882' '1883' '1884'
 '1885' '1886' '1887' '1888' '1889' '1890' '1891' '1892' '1893' '1894'
 '1895' '1896' '1897' '1898' '1899' '1900' '1901' '1902' '1903' '1904'
 '1905' '1906' '1907' '1908' '1909' '1910' '1911' '1912' '1913' '1914'
 '1915' '1916' '1917' '1918' '1919' '1920' '1921' '1922' '1923' '1924'
 '1925' '1926' '1927' '1928' '1929' '1930' '1931' '1932' '1933' '1934'
 '1935' '1936' '1937' '1938' '1939' '1940' '1941' '1942' '1943' '1944'
 '1945' '1946' '1947' '1948' '1949' '1950' '1951' '1952' '1953' '1954'
 '1955' '1956' '1957' '1958' '1959' '1960' '1961' '1962' '1963' '1964'
 '1965' '1966' '1967' '1968' '1969' '1970' '1971' '1972' '1973' '1974'
 '1975' '1976' '1977' '1978' '1979' '1980' '1981' '1982' '1983' '1984'
 '1985' '1986' '1987' '1988' '1989' '1990' '1991' '1992' '1993' '1994'
 '1995' '1996' '1997' '1998' '1999' '2000' '2001' '2002' '2003' '2004'
 '2005' '2006' '2007' '2008' '2009' '2010' '2011' '2012' '2013' '2014'
 '20

In [19]:


#turn df into a dictionary for processing into mongo
climate_dict = climate_df.to_dict('list')

In [20]:
climate_df.to_excel('C:/Users/joelb/Documents/Github/ClimateChange/test.xlsx')

In [24]:
climate_df = pd.read_excel('C:/Users/joelb/Documents/Github/ClimateChange/test.xlsx')

climate_dict = climate_df.to_dict('list')



In [26]:
climate_dict['Lat']

[24.453884,
 9.0764785,
 5.6037168,
 -25.0662897,
 8.9806034,
 36.753768,
 -19.0553711,
 31.9453666,
 52.3702157,
 42.50631740000001,
 39.9333635,
 -18.8791902,
 -13.8506958,
 37.9600766,
 15.3228767,
 51.16052269999999,
 -25.2637399,
 37.9838096,
 -21.2129007,
 33.3128057,
 40.40926169999999,
 12.6392316,
 4.903052199999999,
 13.7563309,
 4.3946735,
 13.4548761,
 17.3026058,
 39.90419989999999,
 33.8937913,
 44.786568,
 17.2510114,
 52.52000659999999,
 46.9479739,
 42.8746212,
 11.8632196,
 -29.1155566,
 4.710988599999999,
 -15.826691,
 48.1485965,
 -4.2633597,
 13.0968511,
 50.8503463,
 44.4267674,
 47.497912,
 -34.6036844,
 -3.361378,
 30.0444196,
 -35.2809368,
 -33.9264168,
 10.4805937,
 14.0101094,
 4.92242,
 18.3419004,
 47.0104529,
 9.641185499999999,
 55.6760968,
 14.716677,
 33.5138073,
 23.810332,
 -8.5568557,
 11.825138,
 -6.162959000000001,
 25.2854473,
 54.1523372,
 53.3498053,
 38.5597722,
 -37.0680421,
 27.1500384,
 34.684073,
 -10.4318725,
 8.4656765,
 -8.5211471,
 -24.

In [27]:
climate_dict = {'City': climate_dict['City'],
               'Country':climate_dict['Country'],
               'Lat':climate_dict['Lat'],
               'Lng':climate_dict['Lng'],
                'rng_1880_1910': climate_dict['rng_1880_1910'],
                'rng_1911_1940':climate_dict['rng_1911_1940'],
                'rng_1941_1970':climate_dict['rng_1941_1970'],
                'rng_1971_2000':climate_dict['rng_1971_2000'],
                'rng_2001_2017':climate_dict['rng_2001_2017']
               }


In [28]:
climate_dict['Lat']

[24.453884,
 9.0764785,
 5.6037168,
 -25.0662897,
 8.9806034,
 36.753768,
 -19.0553711,
 31.9453666,
 52.3702157,
 42.50631740000001,
 39.9333635,
 -18.8791902,
 -13.8506958,
 37.9600766,
 15.3228767,
 51.16052269999999,
 -25.2637399,
 37.9838096,
 -21.2129007,
 33.3128057,
 40.40926169999999,
 12.6392316,
 4.903052199999999,
 13.7563309,
 4.3946735,
 13.4548761,
 17.3026058,
 39.90419989999999,
 33.8937913,
 44.786568,
 17.2510114,
 52.52000659999999,
 46.9479739,
 42.8746212,
 11.8632196,
 -29.1155566,
 4.710988599999999,
 -15.826691,
 48.1485965,
 -4.2633597,
 13.0968511,
 50.8503463,
 44.4267674,
 47.497912,
 -34.6036844,
 -3.361378,
 30.0444196,
 -35.2809368,
 -33.9264168,
 10.4805937,
 14.0101094,
 4.92242,
 18.3419004,
 47.0104529,
 9.641185499999999,
 55.6760968,
 14.716677,
 33.5138073,
 23.810332,
 -8.5568557,
 11.825138,
 -6.162959000000001,
 25.2854473,
 54.1523372,
 53.3498053,
 38.5597722,
 -37.0680421,
 27.1500384,
 34.684073,
 -10.4318725,
 8.4656765,
 -8.5211471,
 -24.